In [1]:
# import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load datasets
sales = pd.read_csv("../data/sales.csv", index_col=0) # offline sales
online = pd.read_csv("../data/online.csv", index_col=0) # online sales
markdowns = pd.read_csv("../data/markdowns.csv", index_col=0) # products sold at lower prices
price_history = pd.read_csv("../data/price_history.csv", index_col=0) # change in product prices
discounts_history = pd.read_csv("../data/discounts_history.csv", index_col=0) # details about discounts
actual_matrix = pd.read_csv("../data/actual_matrix.csv", index_col=0) 
catalog = pd.read_csv("../data/catalog.csv", index_col=0) # product catalog
stores = pd.read_csv("../data/stores.csv", index_col=0) # store location 

In [23]:
discounts_history.head(5)

,date,item_id,sale_price_before_promo,sale_price_time_promo,promo_type_code,doc_id,number_disc_day,store_id
3648316,2022-08-28,cde97a9ec3ef,729.9,669.9,6.0,89ea9f0c-1c08-11ed-ad4d-005056861040,13.0,1
3648317,2022-08-28,2906fbc9e11c,129.9,99.9,NaN,039e85df-1e51-11ed-ad4d-005056861040,11.0,1
3648318,2022-08-28,c48c3a3d8c01,129.9,99.9,NaN,039e85df-1e51-11ed-ad4d-005056861040,11.0,1
3648319,2022-08-28,f85243ca61e6,129.9,99.9,NaN,039e85df-1e51-11ed-ad4d-005056861040,11.0,1
3648320,2022-08-28,27ce75534065,129.9,99.9,NaN,039e85df-1e51-11ed-ad4d-005056861040,11.0,1


In [25]:
price_history.head(5)

,date,item_id,price,code,store_id
0,2023-08-01,4a1f95fe4d4b,499.9,1,1
1,2023-08-01,bf43a9d3ae13,0.0,9999999999,1
2,2023-08-01,0dd8744decce,129.9,1,1
3,2023-08-01,9b38c7e4dba9,136.0,1,1
4,2023-08-01,b791e8c9347c,0.0,1,1


In [21]:
# Add source identifiers
sales['source'] = "offline"
online['source'] = "online"

# Convert dates to datetime format
sales['date'] = pd.to_datetime(sales['date'])
online['date'] = pd.to_datetime(online['date'])

# Rename columns for clarity
sales = sales.rename(columns={"quantity": "quantity_offline", "price_base": "price_base_offline", "sum_total": "sum_total_offline"})
online = online.rename(columns={"quantity": "quantity_online", "price_base": "price_base_online", "sum_total": "sum_total_online"})

# Define merge keys
key_cols = ["date", "item_id", "store_id","source"]

# Merge datasets
data = pd.merge(sales, online, how="outer", on=key_cols)

# Inspect the result
print(data.head())
print(f"Shape of merged data: {data.shape}")


        date       item_id  quantity_offline  price_base_offline  \
0 2022-08-28  001829cb707d               7.0              134.76   
1 2022-08-28  001829cb707d               1.0              148.00   
2 2022-08-28  0022b986c8f0               2.0               59.90   
3 2022-08-28  00274a69c705               1.0               35.90   
4 2022-08-28  00274a69c705               5.0               35.90   

   sum_total_offline  store_id offline_sales   source  quantity_online  \
0             943.32         1       offline  offline              NaN   
1             148.00         2       offline  offline              NaN   
2             119.80         1       offline  offline              NaN   
3              35.90         2       offline  offline              NaN   
4             179.50         3       offline  offline              NaN   

   price_base_online  sum_total_online online_sales  
0                NaN               NaN          NaN  
1                NaN               NaN

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8556097 entries, 0 to 8556096
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   item_id             object        
 2   quantity_offline    float64       
 3   price_base_offline  float64       
 4   sum_total_offline   float64       
 5   store_id            int64         
 6   offline_sales       object        
 7   source              object        
 8   quantity_online     float64       
 9   price_base_online   float64       
 10  sum_total_online    float64       
 11  online_sales        object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 783.3+ MB


In [27]:
data

,date,item_id,quantity_offline,price_base_offline,sum_total_offline,store_id,offline_sales,source,quantity_online,price_base_online,sum_total_online,online_sales
0,2022-08-28,001829cb707d,7.000,134.76,943.32,1,offline,offline,NaN,NaN,NaN,NaN
1,2022-08-28,001829cb707d,1.000,148.00,148.00,2,offline,offline,NaN,NaN,NaN,NaN
2,2022-08-28,0022b986c8f0,2.000,59.90,119.80,1,offline,offline,NaN,NaN,NaN,NaN
3,2022-08-28,00274a69c705,1.000,35.90,35.90,2,offline,offline,NaN,NaN,NaN,NaN
4,2022-08-28,00274a69c705,5.000,35.90,179.50,3,offline,offline,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8556092,2024-09-26,ffea55e3589f,0.760,499.89,379.92,1,offline,offline,NaN,NaN,NaN,NaN
8556093,2024-09-26,ffea55e3589f,0.105,719.14,75.51,2,offline,offline,NaN,NaN,NaN,NaN
8556094,2024-09-26,ffea55e3589f,0.499,499.90,249.45,4,offline,offline,NaN,NaN,NaN,NaN
8556095,2024-09-26,fffb04d8e873,4.000,103.97,415.88,1,offline,offline,NaN,NaN,NaN,NaN


In [4]:
catalog.head()

,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
0,da17e2d5feda,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,БУМАЖНАЯ ПРОДУКЦИЯ,ВЛАЖНЫЕ САЛФЕТКИ,NaN,150.0,NaN,NaN
1,614de2b96018,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
2,0c1f1f3e3e11,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,NaN,NaN,NaN
3,71a7fa99f005,ТОВАРЫ ДЛЯ ДОМА,ПОСУДА,КУХОННАЯ ПОСУДА,NaN,NaN,NaN,NaN
4,ec1bd4d59fe9,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN


In [ ]:
sales.head()

In [ ]:
# Step 1: Merge `sales` with `catalog`
merged_df = pd.merge(sales, catalog, on="item_id", how="left")

In [ ]:
merged_df.head()

In [ ]:
markdowns.head()

In [ ]:
# Step 2: Add `markdowns` data
merged_df = pd.merge(merged_df, markdowns, on=["date", "item_id", "store_id"], how="left")

In [ ]:
merged_df.head()

In [ ]:
# Step 3: Add `price_history` data
merged_df = pd.merge(merged_df, price_history, on=["date", "item_id", "store_id"], how="left")

In [ ]:
merged_df

In [ ]:




# Step 4: Add `discounts_history` data
merged_df = pd.merge(merged_df, discounts_history, on=["date", "item_id", "store_id"], how="left")

# Step 5: Add `stores` data
merged_df = pd.merge(merged_df, stores, on="store_id", how="left")

# Step 6: Filter with `actual_matrix` for available products
filtered_matrix = actual_matrix[["item_id", "store_id", "date"]]
merged_df = pd.merge(merged_df, filtered_matrix, on=["item_id", "store_id", "date"], how="inner")

# Step 7: Optionally add `online` sales data
online = online.rename(columns={"quantity": "quantity_online", "sum_total": "sum_total_online"})
merged_df = pd.merge(merged_df, online, on=["date", "item_id", "store_id"], how="left")

# Final Result
print(merged_df.head())
print(f"Shape of final dataframe: {merged_df.shape}")

# Save the final dataframe to a CSV
merged_df.to_csv("merged_sales_data.csv", index=False)